<a href="https://colab.research.google.com/github/Banafshehkh/Kaggle-Competition/blob/main/Final_Notebook_BK_SG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Authors: Banafsheh Khazali, Shokoofa Ghods
#Date: March 09, 2023

# **GoogleNet**

In [ ]:
# -*- coding: utf-8 -*-
"""Classifier_GoogleNet.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Fy0gzrWHTIPRjUkSrgqG4DEmMwOVy8xF
"""

# Author: Banafsheh Khazali and Shokoofa Ghods
# Date: March 06, 2023

"""## Libraries"""

import os
from os import listdir
from os.path import join, isfile
from PIL import Image

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import random_split

from google.colab import drive

drive.mount('/content/drive')

# !unzip -q "/content/drive/MyDrive/Data/Dataset.zip" -d "/content/drive/MyDrive/Dataset"

"""# Load the csv file containing the image file names and corresponding labels"""

data = pd.read_csv('/content/drive/MyDrive/train.csv')
data

np_data = data.values

"""# Define Transformers"""

from torchvision.transforms import transforms

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-30, 30))], p = 0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),
])

"""# Map images to their labels"""

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # Get the filename and label for the current index
        filename = self.data[index][0]
        label = self.data[index][1]
        
        # Load the image and apply the transformations
        image = Image.open('/content/drive/MyDrive/train/' + filename)
        if self.transform:
            image = self.transform(image)
            label = torch.tensor(label)
        
        # Return the image and label as tensors
        return image, label

    def number_of_classes(self):
      return self.data[:,1].max() + 1

def visualization(image, label):
  plt.figure(figsize= (10,8))
  plt.imshow(image.permute(1,2,0))
  plt.title(label.item())
  plt.xticks([])
  plt.yticks([])
  plt.show()

"""# Load the train dataset from the folder"""

train_dataset = CustomDataset(np_data, transform=train_transform)

"""# Split the data into train and validation sets """

n_total = len(train_dataset)
n_train = int(n_total * 0.8)
n_val = n_total - n_train
train_data, val_data = random_split(train_dataset, [0.8, 0.2])

"""## Visualize one sample"""

img, label = train_dataset[100]
print(img.shape)
visualization(img, label)

"""# Create data loaders for the train and validation sets


"""

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
)

val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
)

"""# Define the neural network model architecture"""

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.googlenet(weights=torchvision.models.GoogLeNet_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, train_dataset.number_of_classes())
model.to(device)

"""# Define loss and optimizer"""

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(device)

"""# Train the model"""

def evaluate(model, loader, device, criterion, mode='validation'):
  model.eval()
  total_correct = 0
  total_loss = 0
  total = 0
  for i, (images, labels) in enumerate(loader):
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, predictions = outputs.max(1)
      total_correct += (labels == predictions).sum()
  loss = total_loss / total
  accuracy = total_correct / total
  print(f'{mode} epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')

# model = model.to(device)
epochs = 20
for epoch in range(epochs):
  model.train()
  total = 0
  total_correct = 0
  total_loss = 0
  total_l = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    total += images.size(0)
    _, predictions = outputs.max(1)
    total_correct += (predictions == labels).sum()
    total_loss += loss.item() * images.size(0)
  accuracy = total_correct / total
  loss = total_loss / total
  print(f'Train epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')
  evaluate(model, val_loader, device, criterion, mode='valid')
  print('---------')
  torch.save(model.state_dict(), f'model_v4_{epoch}.pt')

torch.save(model.state_dict(), 'googlenet.pt')



#ResNet18

In [ ]:
# -*- coding: utf-8 -*-
"""Classifier_ResNet18.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1q4iDBgYax7aH96Bb5sO10TR4FS1Q_95z
"""

# Author: Banafsheh Khazali & Shokoofa Ghods
# Date: March 06, 2023

"""## Libraries"""

import os
from os import listdir
from os.path import join, isfile
from PIL import Image

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import random_split

from google.colab import drive
drive.mount('/content/drive')

# !unzip -q "/content/drive/MyDrive/data/Dataset.zip" -d "data"

data = pd.read_csv('/content/drive/MyDrive/train.csv')
np_data = data.values

filename = '/content/drive/MyDrive/train/'
X = np.empty(shape=(4135,224,224,3), dtype= np.uint8)
for i in range(4135):
  image = Image.open(filename + np_data[i][0]).resize((224,224))
  img = np.array(image)
  X[i] = np.array(image)

"""# Define Transformers"""

from torchvision.transforms import transforms

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-30, 30))], p = 0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.3),
    transforms.RandomGrayscale(0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.2),
    # transforms.RandomInvert(0.2),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
    transforms.RandomApply([transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2))], p=0.2),
])

"""# Map images to their labels"""

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, transform=None):
        self.data = data
        self.transform = transform
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        image = self.x[index]
        label = self.y[index]
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label)
        
        # Return the image and label as tensors
        return image, label

    def number_of_classes(self):
      return self.y.max() + 1

def visualization(image, label=0):
  plt.figure(figsize= (10,8))
  plt.imshow(image.permute(1,2,0))
  plt.xticks([])
  plt.yticks([])
  plt.show()

"""# Load the train dataset from the folder"""

train_dataset = CustomDataset(X,np_data[:,1] , transform=train_transform)

"""# Split the data into train and validation sets """

train_data, val_data = random_split(train_dataset, [0.8, 0.2])

"""## Visualize one sample"""

img, label = train_dataset[106]
print(img.shape)
visualization(img)

"""# Create data loaders for the train and validation sets


"""

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
    
)

"""# Define the neural network model architecture"""

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 100)
model.to(device)

# Fine-tune the pre-trained model
for param in model.parameters():
    param.requires_grad = False
for param in model.layer4.parameters():
    param.requires_grad = True
model.fc.requires_grad = True

"""# Define loss and optimizer"""

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(device)

"""# Train the model"""

def evaluate(model, loader, device, criterion, mode='validation'):
  model.eval()
  total_correct = 0
  total_loss = 0
  total = 0
  for i, (images, labels) in enumerate(loader):
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, predictions = outputs.max(1)
      total_correct += (labels == predictions).sum()
  loss = total_loss / total
  accuracy = total_correct / total
  print(f'{mode} epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')



# model = model.to(device)
epochs = 20
for epoch in range(epochs):
  model.train()
  total = 0
  total_correct = 0
  total_loss = 0
  total_l = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    total += images.size(0)
    _, predictions = outputs.max(1)
    total_correct += (predictions == labels).sum()
    total_loss += loss.item() * images.size(0)
  accuracy = total_correct / total
  loss = total_loss / total
  print(f'Train epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')
  evaluate(model, val_loader, device, criterion, mode='valid')
  print('---------')
  torch.save(model.state_dict(), f'model_v4_{epoch}.pt')

torch.save(model.state_dict(), 'resnet18.pt')

#DenseNet

In [ ]:
# -*- coding: utf-8 -*-
"""Classifier_DenseNet_finetune.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1zdWCJTk4LoluOKm70smMH1LiLY8M4ciU
"""

# Author: Banafsheh Khazali & Shokoofa Ghods
# Date: March 06, 2023

"""## Libraries"""

import os
from os import listdir
from os.path import join, isfile
from PIL import Image

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import random_split

from google.colab import drive
drive.mount('/content/drive')

# !unzip -q "/content/drive/MyDrive/data/Dataset.zip" -d "data"

data = pd.read_csv('/content/drive/MyDrive/train.csv')
np_data = data.values

filename = '/content/drive/MyDrive/train/'
X = np.empty(shape=(4135,224,224,3), dtype= np.uint8)
for i in range(4135):
  image = Image.open(filename + np_data[i][0]).resize((224,224))
  img = np.array(image)
  X[i] = np.array(image)

"""# Define Transformers"""

from torchvision.transforms import transforms

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-30, 30))], p = 0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.3),
    transforms.RandomGrayscale(0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.2),
    # transforms.RandomInvert(0.2),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
    transforms.RandomApply([transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2))], p=0.2),
])

"""# Map images to their labels"""

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, transform=None):
        self.data = data
        self.transform = transform
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        image = self.x[index]
        label = self.y[index]
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label)
        
        # Return the image and label as tensors
        return image, label

    def number_of_classes(self):
      return self.y.max() + 1

def visualization(image, label=0):
  plt.figure(figsize= (10,8))
  plt.imshow(image.permute(1,2,0))
  plt.xticks([])
  plt.yticks([])
  plt.show()

"""# Load the train dataset from the folder"""

train_dataset = CustomDataset(X,np_data[:,1] , transform=train_transform)

"""# Split the data into train and validation sets """

train_data, val_data = random_split(train_dataset, [0.8, 0.2])

"""## Visualize one sample"""

img, label = train_dataset[106]
print(img.shape)
visualization(img)

"""# Create data loaders for the train and validation sets


"""

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
    
)

"""# Define the neural network model architecture"""

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = torchvision.models.densenet121(pretrained=True)

# Replace last fully connected layer with a new one
num_ftrs = model.classifier.in_features
model.classifier = torch.nn.Linear(num_ftrs, train_dataset.number_of_classes())
model.to(device)

# """## FineTuning

# we first freeze all layers except for the last one, and trains only the last layer for a few epochs. The we unfreeze all layers, and train the whole model for more epochs.
# """

# # Freeze all layers except for the last one
# for param in model.parameters():
#     param.requires_grad = True
# model.classifier.requires_grad = True

# Define loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# # Train the last layer
# num_epochs = 20
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         inputs, labels = data
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# # Unfreeze all layers
# for param in model.parameters():
#     param.requires_grad = True

"""# Define loss and optimizer"""

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(device)

"""# Train the model"""

def evaluate(model, loader, device, criterion, mode='validation'):
  model.eval()
  total_correct = 0
  total_loss = 0
  total = 0
  for i, (images, labels) in enumerate(loader):
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, predictions = outputs.max(1)
      total_correct += (labels == predictions).sum()
  loss = total_loss / total
  accuracy = total_correct / total
  print(f'{mode} epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')

# model = model.to(device)
epochs = 30
for epoch in range(epochs):
  model.train()
  total = 0
  total_correct = 0
  total_loss = 0
  total_l = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    total += images.size(0)
    _, predictions = outputs.max(1)
    total_correct += (predictions == labels).sum()
    total_loss += loss.item() * images.size(0)
  accuracy = total_correct / total
  loss = total_loss / total
  print(f'Train epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')
  evaluate(model, val_loader, device, criterion, mode='valid')
  print('---------')
  torch.save(model.state_dict(), f'model_v4_{epoch}.pt')



torch.save(model.state_dict(), 'densenet.pt')

#DenseNet with FineTuning



In [ ]:
# -*- coding: utf-8 -*-
"""Classifier_DenseNet_finetune (1).ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1MXh8gCj8bzR3t9SaP2CmGyOEGwXJjJ1w
"""

# Author: Banafsheh Khazali & Shokoofa Ghods
# Date: March 06, 2023

"""## Libraries"""

import os
from os import listdir
from os.path import join, isfile
from PIL import Image

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

# !unzip -q "/content/drive/MyDrive/data/Dataset.zip" -d "data"

data = pd.read_csv('data/train.csv')
np_data = data.values
Y = np_data[:,1]

filename = 'data/'
X = np.empty(shape=(4135,224,224,3), dtype= np.uint8)
for i in range(4135):
  image = Image.open(filename + np_data[i][0]).resize((224,224))
  img = np.array(image)
  X[i] = np.array(image)

"""# Define Transformers"""

from torchvision.transforms import transforms

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-30, 30))], p = 0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.3),
    transforms.RandomGrayscale(0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.2),
    # transforms.RandomInvert(0.2),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
    transforms.RandomApply([transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2))], p=0.2),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.1),
    # transforms.RandomVerticalFlip(0.1),
    # transforms.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
    # transforms.RandomApply([transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2))], p=0.2),
])

"""# Map images to their labels"""

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, transform=None):
        self.data = data
        self.transform = transform
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        image = self.x[index]
        label = self.y[index]
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label)
        
        # Return the image and label as tensors
        return image, label

    def number_of_classes(self):
      return self.y.max() + 1

def visualization(image, label=0):
  plt.figure(figsize= (10,8))
  plt.imshow(image.permute(1,2,0))
  plt.xticks([])
  plt.yticks([])
  plt.show()

"""# Split the data into train and validation sets """

#since the augmentation for each data is different we split first then pass it to the dataset 
x_train, x_val, y_train, y_val = train_test_split(X , Y, test_size=0.2, random_state=42)

"""# Load the train dataset from the folder"""

train_dataset = CustomDataset(x_train, y_train , transform=train_transform)
val_dataset = CustomDataset(x_val, y_val , transform=val_transform)

"""## Visualize one sample"""

img, label = val_dataset[106]
print(img.shape)
visualization(img)

"""# Create data loaders for the train and validation sets


"""

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
    
)

"""# Define the neural network model architecture"""

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = torchvision.models.densenet121(pretrained=True, )

# Replace last fully connected layer with a new one
num_ftrs = model.classifier.in_features
model.classifier = torch.nn.Linear(num_ftrs, train_dataset.number_of_classes())
model.to(device)

"""## FineTuning

we first freeze all layers except for the last one, and trains only the last layer for a few epochs. The we unfreeze all layers, and train the whole model for more epochs.
"""

list(model.classifier.parameters())

# Freeze all layers except for the last one
for param in list(model.classifier.parameters()):
    param.requires_grad = True
# model.classifier.requires_grad = True

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)

# Train the last layer
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Unfreeze all layers, in this phase we train the model again to fune tune parameters while we have preserved the parameters model has been learned during the last pre-training
for param in model.parameters():
    param.requires_grad = True

"""# Define loss and optimizer"""

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(device)

"""# Train the model"""

def evaluate(model, loader, device, criterion, mode='validation'):
  model.eval()
  total_correct = 0
  total_loss = 0
  total = 0
  for i, (images, labels) in enumerate(loader):
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, predictions = outputs.max(1)
      total_correct += (labels == predictions).sum()
  loss = total_loss / total
  accuracy = total_correct / total
  print(f'{mode} epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')

# model = model.to(device)
epochs = 20
for epoch in range(epochs):
  model.train()
  total = 0
  total_correct = 0
  total_loss = 0
  total_l = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    total += images.size(0)
    _, predictions = outputs.max(1)
    total_correct += (predictions == labels).sum()
    total_loss += loss.item() * images.size(0)
  accuracy = total_correct / total
  loss = total_loss / total
  print(f'Train epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')
  evaluate(model, val_loader, device, criterion, mode='valid')
  print('---------')
  torch.save(model.state_dict(), f'model_v4_{epoch}.pt')



torch.save(model.state_dict(), 'dense_finetune.pt')

#**EffiecientNet**

In [ ]:
# -*- coding: utf-8 -*-
"""Classifier_EfficientNet.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1DeVfcwE5M4dT1WM6D6s05PNDS89Up2ax
"""

# Author: Banafsheh Khazali and Shokoofa Ghods
# Date: March 06, 2023

"""## Libraries"""

!pip install git+https://github.com/lukemelas/EfficientNet-PyTorch

import os
from os import listdir
from os.path import join, isfile
from PIL import Image
from datetime import datetime

import numpy as np
import pandas as pd
# import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import random_split
import torchvision.models as models
from efficientnet_pytorch import EfficientNet

from google.colab import drive
drive.mount('/content/drive')

# !unzip -q "/content/drive/MyDrive/data/Dataset.zip" -d "data"

data = pd.read_csv('/content/drive/MyDrive/train.csv')
data
np_data = data.values

"""## Load all data in a numpy array"""

filename = '/content/drive/MyDrive/train/'
X = np.empty(shape=(4135,224,224,3), dtype= np.uint8)
for i in range(4135):
  image = Image.open(filename + np_data[i][0]).resize((224,224))
  img = np.array(image)
  X[i] = np.array(image)

"""# Define Transformers"""

from torchvision.transforms import transforms

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-30, 30))], p = 0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.3),
    transforms.RandomGrayscale(0.2),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.2),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
    transforms.RandomApply([transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2))], p=0.2),
])

"""# Map images to their labels"""

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, transform=None):
        self.data = data
        self.transform = transform
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        image = self.x[index]
        label = self.y[index]
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label)
        
        # Return the image and label as tensors
        return image, label

    def number_of_classes(self):
      return self.y.max() + 1

def visualization(image, label=0):
  plt.figure(figsize= (10,8))
  plt.imshow(image.permute(1,2,0))
  plt.xticks([])
  plt.yticks([])
  plt.show()

"""# Load the train dataset from the folder"""

train_dataset = CustomDataset(X,np_data[:,1] , transform=train_transform)

"""# Split the data into train and validation sets """

train_data, val_data = random_split(train_dataset, [0.8, 0.2])

"""## Visualize one sample"""

img, label = train_dataset[106]
print(img.shape)
visualization(img)

"""# Create data loaders for the train and validation sets


"""

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    val_data,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
    
)

"""# Using EfficientNet"""

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load EfficientNet model with pretrained weights
model = EfficientNet.from_pretrained('efficientnet-b0')

# Replace last fully connected layer with a new one
num_ftrs = model._fc.in_features
model._fc = torch.nn.Linear(num_ftrs, train_dataset.number_of_classes())

model.to(device)

"""# Define loss and optimizer"""

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(device)

"""# Train the model"""

def evaluate(model, loader, device, criterion, mode='validation'):
  model.eval()
  total_correct = 0
  total_loss = 0
  total = 0
  for i, (images, labels) in enumerate(loader):
    images = images.to(device)
    labels = labels.to(device)
    with torch.no_grad():
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, predictions = outputs.max(1)
      total_correct += (labels == predictions).sum()
  loss = total_loss / total
  accuracy = total_correct / total
  print(f'{mode} epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')

# model = model.to(device)
epochs = 20
for epoch in range(epochs):
  model.train()
  total = 0
  total_correct = 0
  total_loss = 0
  total_l = 0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    total += images.size(0)
    _, predictions = outputs.max(1)
    total_correct += (predictions == labels).sum()
    total_loss += loss.item() * images.size(0)
  accuracy = total_correct / total
  loss = total_loss / total
  print(f'Train epoch {epoch+1}: Loss({loss:6.4f}) Accuracy ({accuracy:6.4f})')
  evaluate(model, val_loader, device, criterion, mode='valid')
  print('---------')
  torch.save(model.state_dict(), f'model_v4_{epoch}.pt')

torch.save(model.state_dict(), 'efficient.pt')

# **Ensemble Learning**

In [ ]:
l = []

for i in range(3):
  l.append(torch.tensor(pd.read_csv(f'prediction{i+1}.csv')['predicted'], dtype= float))
en = torch.mode(torch.stack(l), dim = 0)
for in range(len(predictions)):
  predictions[i]['predicted'] = int(en.values[i].item())

# **Predictions**

In [ ]:
torch.save(model, "/content/drive/MyDrive/best_eff.pt")
model = torch.load('/content/drive/MyDrive/best_eff.pt')
test_csv = pd.read_csv('/content/drive/MyDrive/test.csv')

test_dir = '/content/drive/MyDrive/test/'
test_files = os.listdir(test_dir)

model.eval()
from torchvision.transforms import ToTensor
predictions = []

# Define data transformation pipeline for test set
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
  
])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


with torch.no_grad():
    for file in test_files:
        image_path = os.path.join(test_dir, file)
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image)
        # image_tensor = ToTensor()(image_tensor)
        image_tensor = image_tensor.to(device) 
        output = model(image_tensor.unsqueeze(0))
        _, predicted = torch.max(output.data, 1)
        predictions.append({'id': f"test/{file}",
                            'predicted': predicted.item()})
        

# Save predictions in CSV file
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv('/content/drive/MyDrive/predictions.csv', index=False)